In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization
import time
import quandl
import matplotlib.pyplot as plt
from timeseries import generate_time_series
from sklearn.utils import shuffle
from sklearn import preprocessing

In [2]:
x1 = np.genfromtxt('datas/x_nasdaq_high_timeseries8.txt')
x2 = np.genfromtxt('datas/x_nasdaq_low_timeseries8.txt')
x3 = np.genfromtxt('datas/x_nasdaq_open_timeseries8.txt')
x4 = np.genfromtxt('datas/x_nasdaq_close_timeseries8.txt')
y = np.genfromtxt('datas/y_nasdaq_open_timeseries8.txt')

In [3]:
x_flat = np.concatenate([x1,x2,x3,x4], -1)
print(x_flat.shape)

x1 = x1.reshape(x1.shape[0], x1.shape[1], 1)
x2 = x2.reshape(x2.shape[0], x2.shape[1], 1)
x3 = x3.reshape(x3.shape[0], x3.shape[1], 1)
x4 = x4.reshape(x4.shape[0], x4.shape[1], 1)
x = np.concatenate([x1,x2, x3, x4],-1)
print(x.shape)

(528497, 32)
(528497, 8, 4)


In [3]:
NAME = f"nasdaq8-32+96+32{int(time.time())}"

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
ctrl = Sequential()
ctrl.add(Dense(32, activation='relu', input_shape=(8,)))
ctrl.add(Dropout(0.2))
ctrl.add(BatchNormalization())
ctrl.add(Dense(96, activation='relu'))
ctrl.add(Dropout(0.2))
ctrl.add(BatchNormalization())
ctrl.add(Dense(32, activation='relu'))
ctrl.add(Dropout(0.2))
ctrl.add(BatchNormalization())
ctrl.add(Dense(1, activation='relu'))

opt = tf.keras.optimizers.Adam(lr=0.001, decay=5e-7)

ctrl.compile(optimizer=opt,
             loss='mse',
             metrics=['mean_absolute_percentage_error'])

ctrl.fit(x3, y, batch_size=128, epochs=30, callbacks=[tensorboard], validation_split=0.2)

Epoch 1/30
3304/3304 [==============================] - 19s 6ms/step - loss: 0.0544 - mean_absolute_percentage_error: 11.5614 - val_loss: 0.0036 - val_mean_absolute_percentage_error: 4.0910

KeyboardInterrupt: 

In [11]:
NAME = f"nasdaqLSTM2x128+dropout0.4+32{int(time.time())}"

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
model = Sequential()
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(LSTM(128))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='relu'))

opt = tf.keras.optimizers.Adam(lr=0.0005, decay=1e-6)

model.compile(optimizer=opt,
              loss='mse',
              metrics=['mean_absolute_percentage_error'])

model.fit(x, y, batch_size=128, epochs=30, callbacks=[tensorboard], validation_split=0.2)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

3304/3304 [==============================] - 36s 11ms/step - loss: 0.0397 - mean_absolute_percentage_error: 12.9729 - val_loss: 0.0263 - val_mean_absolute_percentage_error: 15.1926
Epoch 2/30
3304/3304 [==============================] - 36s 11ms/step - loss: 0.0031 - mean_absolute_percentage_error: 3.9674 - val_loss: 0.0020 - val_mean_absolute_percentage_error: 2.8216
Epoch 3/30
3304/3304 [==============================] - 36s 11ms/step - loss: 0.0020 - mean_absolute_percentage_error: 2.9680 

In [ ]:
NAME = f"nasdaqLSTM-3x64+dropout0.4+32{int(time.time())}"

tensorboard = TensorBoard(log_dir=f'logs/{NAME}')
model2 = Sequential()
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(LSTM(64, return_sequences=True))
model2.add(Dropout(0.2))
model2.add(BatchNormalization())
model2.add(Dense(32, activation='relu'))
model2.add(Dense(1, activation='relu'))

opt = tf.keras.optimizers.Adam(lr=0.0005, decay=1e-6)

model2.compile(optimizer=opt,
              loss='mse',
              metrics=['mean_absolute_percentage_error'])

model2.fit(x, y, batch_size=128, epochs=30, callbacks=[tensorboard], validation_split=0.2)

Epoch 1/30

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

3304/3304 [==============================] - 34s 10ms/step - loss: 0.0757 - mean_absolute_percentage_error: 12.9430 - val_loss: 0.0060 - val_mean_absolute_percentage_error: 4.9477
Epoch 2/30
3304/3304 [==============================] - 30s 9ms/step - loss: 0.0055 - mean_absolute_percentage_error: 5.0210 - val_loss: 0.0054 - val_mean_absolute_percentage_error: 4.7215
Epoch 3/30
3304/3304 [==============================] - 30s 9ms/step - loss: 0.0051 - mean_absolute_percentage_error: 4.7660 - v